In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup

In [ ]:
# 查询 IEEE 投稿文章的状态 TODO*3

with Browser('chrome', headless=False) as b:
    b.visit('https://mc.manuscriptcentral.com/tmi-ieee/') # TODO: 期刊对应的网址
    # 等待元素出现（最长 10 秒）
    if b.is_element_present_by_name("USERID", wait_time=10):
        b.fill("USERID", "USERID") # TODO: 填写账号
        b.fill("PASSWORD", "PASSWORD") # TODO: 填写密码
        b.find_by_id("logInButton").first.click()
        if b.is_element_present_by_css("a[href*='AUTHOR_VIEW_MANUSCRIPTS']", wait_time=10): 
            b.find_by_css("a[href*='AUTHOR_VIEW_MANUSCRIPTS']").first.click()
            html_obj = b.html
            soup = BeautifulSoup(html_obj,"lxml")
            table = soup.find("span", attrs={"class": "pagecontents"})
            td = soup.find("td", {"data-label": "title"})
            title = td.get_text(separator="\n", strip=True).split("\n")[0]
            print(f"Title: {title}")
            print(f"Status: {table.string}")
            current_manuscript_status = table.string
        else:
            print("❌ Author button not found")
    else:
        print("❌ USERID input not found")

In [ ]:
# 查询 Elsevier 投稿文章的状态 TODO*4
with Browser('chrome', headless=False) as b:
    b.visit("https://www.editorialmanager.com/neucom/mainpage.html") # TODO: 期刊对应的网址

    # 等待 iframe 出现
    if b.is_element_present_by_name("login", wait_time=10):
        # 切换到 iframe
        b.driver.switch_to.frame("login")

        # 等待用户名输入框
        if b.is_element_present_by_name("username", wait_time=10):
            b.fill("username", "username") # TODO: 填写账号
            b.fill("password", "password") # TODO: 填写密码

            try:
                b.find_by_name("authorLogin").first.click()
            except:
                b.execute_script("doLogin('author');")
        else:
            print("❌ username input not found inside iframe")
    else:
        print("❌ login iframe not found")

    # 登录后要切回主页面，否则找不到菜单
    b.driver.switch_to.default_content()
    # 再切换到 content iframe
    b.driver.switch_to.frame("content")

    stage = 'revision' # TODO: 此时是否在revision，如果是，填写 'revision'，如果还在初审，填写 'init'

    # 等待并进入稿件页面
    if stage == 'revision' and b.is_element_present_by_css("a[href*='auth_RevisionsBeingProcessed.asp']", wait_time=10): 
        b.find_by_css("a[href*='auth_RevisionsBeingProcessed.asp']").first.click()
    elif stage == 'init' and b.is_element_present_by_css("a[href*='uth_pendSubmissions.asp']", wait_time=10): #TODO: Submissions Being Processed
        b.find_by_css("a[href*='uth_pendSubmissions.asp']").first.click()
    else:
        print("❌ The stage name is wrong")

    html_obj = b.html
    soup = BeautifulSoup(html_obj, "lxml")

    rows = soup.select("tr[id^='row']")  # 匹配所有行
    for row in rows:
        cols = row.find_all("td")
        title = cols[2].get_text(strip=True)
        status = cols[-1].get_text(strip=True)
        print(f"Title: {title}")
        print(f"Status: {status}")